<a href="https://colab.research.google.com/github/google/business_intelligence_group/blob/development/solutions/causal-impact/CausalImpact_with_Experimental_Design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CausalImpact with Experimental Design**

This Colab file contains *Experimental Design* and *CausalImpact Analysis*.

See [README.md](https://github.com/google/business_intelligence_group/tree/main/solutions/causal-impact) for details

---

Copyright 2024 Google LLC

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

In [ ]:
class DataLoader(object):\n\n    def _load_data_from_sheet(spreadsheet_url, sheet_name):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      """load_data_from_sheet load data from spreadsheet.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      Args:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        spreadsheet_url: Spreadsheet url with data.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        sheet_name: Sheet name with data.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      """
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      auth.authenticate_user()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      creds, _ = default()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      gc = gspread.authorize(creds)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      _workbook = gc.open_by_url(spreadsheet_url)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      _worksheet = _workbook.worksheet(sheet_name)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      df_sheet = pd.DataFrame(_worksheet.get_all_values())
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      df_sheet.columns = list(df_sheet.loc[0, :])
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      df_sheet.drop(0, inplace=True)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      df_sheet.reset_index(drop=True, inplace=True)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      df_sheet.replace(',', '', regex=True, inplace=True)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      df_sheet.rename(columns=lambda x: x.replace(" ", ""), inplace=True)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      df_sheet = df_sheet.apply(pd.to_numeric, errors='ignore')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      return df_sheet
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    def _load_data_from_csv(csv_name):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      """load_data_from_csv read data from csv.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      Args:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      csv_name: csv file name.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      """
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      uploaded = files.upload()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      df_csv = pd.read_csv(io.BytesIO(uploaded[csv_name]))
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      df_csv.replace(',', '', regex=True, inplace=True)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      df_csv.rename(columns=lambda x: x.replace(" ", ""), inplace=True)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      df_csv = df_csv.apply(pd.to_numeric, errors='ignore')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      return df_csv
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    def _load_data_from_bigquery(bq_project_id, bq_table_name):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      """_load_data_from_bigquery load data from bigquery.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      Args:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      bq_project_id: bigquery project id.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      bq_table_name: bigquery table name
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      """
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      auth.authenticate_user()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      client = bigquery.Client(project=bq_project_id)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      query = 'SELECT * FROM `' + bq_table_name + '`;'
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      df_bq = client.query(query).to_dataframe()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      df_bq.replace(',', '', regex=True, inplace=True)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      df_bq.rename(columns=lambda x: x.replace(" ", ""), inplace=True)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      df_bq = df_bq.apply(pd.to_numeric, errors='ignore')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      return df_bq
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n\n

In [ ]:
class DataFormatter(object):\n\n    def format_data(self, orchestrator):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      """Formats the loaded data for causal impact analysis or experimental design.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      This method performs several data transformation steps:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      1. Cleans column names by removing spaces from `date_col`, `pivot_col`, and `kpi_col`.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      2. Converts the data to a wide format if specified by `data_type_selection`.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      3. Drops columns specified in `exclude_cols`.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      4. Converts the date column to datetime objects and sets it as the DataFrame index.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      5. Reindexes the DataFrame to ensure a continuous date range from the minimum to maximum date.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      6. Calculates `tick_count` for visualization purposes.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      7. Provides visual feedback on the data formatting success or failure.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      8. Displays an overview of the formatted data, including index, date range, and missing values.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      9. Visualizes data trends (total and individual) and descriptive statistics.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      Raises:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          Exception: If any error occurs during data formatting, often due to
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                     mismatched data format selection (wide/narrow) or incorrect
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                     column names. Provides specific error messages to guide debugging.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      """
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      orchestrator.date_col_name = self.date_col.value.replace(' ', '')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      self.pivot_col_name = self.pivot_col.value.replace(' ', '')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      self.kpi_col_name = self.kpi_col.value.replace(' ', '')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      try:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        if self.data_type_selection.selected_index == 0:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          orchestrator.formatted_data =\1orchestrator.\2.copy()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        elif self.data_type_selection.selected_index == 1:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          orchestrator.formatted_data = self._shape_wide(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n             \1orchestrator.\2,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              orchestrator.date_col_name,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              self.pivot_col_name,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              self.kpi_col_name,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        orchestrator.formatted_data.drop(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            self.exclude_cols.value.replace(', ', ',').split(','),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            axis=1,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            errors='ignore',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            inplace=True,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        orchestrator.formatted_data[orchestrator.date_col_name] = pd.to_datetime(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.formatted_data[orchestrator.date_col_name]
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        orchestrator.formatted_data = orchestrator.formatted_data.set_index(orchestrator.date_col_name)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        orchestrator.formatted_data = orchestrator.formatted_data.reindex(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            pd.date_range(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                start=orchestrator.formatted_data.index.min(),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                end=orchestrator.formatted_data.index.max(),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                name=orchestrator.formatted_data.index.name))
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        orchestrator.tick_count = len(orchestrator.formatted_data.resample('M')) - 1
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        self._apply_text_style(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            'success',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            '
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    def _shape_wide(dataframe, date_column, pivot_column, kpi_column):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      """shape_wide pivots the data in the specified column.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      Converts long data to wide data suitable for experiment design.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      Args:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          dataframe: The DataFrame to be pivoted.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          date_column: The name of the column that contains the dates.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          pivot_column: The name of the column that contains the pivot keys.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          kpi_column: The name of the column that contains the KPI values.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      Returns:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          A DataFrame with the pivoted data.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      """
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      # Check if the pivot_column is a single column or a list of columns.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      if ',' in pivot_column:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        group_cols = pivot_column.replace(', ', ',').split(',')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      else:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        group_cols = [pivot_column]
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      pivoted_df = pd.pivot_table(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          (dataframe[[date_column] + [kpi_column] + group_cols])
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          .groupby([date_column] + group_cols)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          .sum(),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          index=date_column,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          columns=group_cols,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          fill_value=0,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      # Drop the first level of the column names.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      pivoted_df.columns = pivoted_df.columns.droplevel(0)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      # If there are multiple columns, convert the column names to a single string.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      if len(pivoted_df.columns.names) > 1:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        new_cols = [
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            '_'.join([x.replace(',', '_') for x in y])
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            for y in pivoted_df.columns.values
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        ]
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        pivoted_df.columns = new_cols
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      pivoted_df = pivoted_df.reset_index()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      return pivoted_df
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n\n

In [ ]:
class ExperimentalDesigner(object):\n\n    def run_experimental_design(self, orchestrator):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      if orchestrator.date_selection.selected_index == 0:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        orchestrator.start_date_value = min(orchestrator.formatted_data.index).date()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        orchestrator.end_date_value = max(orchestrator.formatted_data.index).date()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      else:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        orchestrator.start_date_value = orchestrator.start_date.value
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        orchestrator.end_date_value = orchestrator.end_date.value
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      if orchestrator.design_type.selected_index == 0:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        orchestrator.distance_data = self._n_part_split(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.formatted_data.query(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                '@orchestrator.start_date_value <= index <= @orchestrator.end_date_value'
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.num_of_split.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            CausalImpactAnalysis.NUM_OF_ITERATION
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      elif orchestrator.design_type.selected_index == 1:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        orchestrator.distance_data = self._find_similar(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.formatted_data.query(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                '@orchestrator.start_date_value <= index <= @orchestrator.end_date_value'
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.target_columns.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.num_of_pick_range.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.num_of_covariate.value
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      elif orchestrator.design_type.selected_index == 2:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        orchestrator.distance_data = self._from_share(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.formatted_data.query(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                '@orchestrator.start_date_value <= index <= @orchestrator.end_date_value'
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.target_share.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      elif orchestrator.design_type.selected_index == 3:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        orchestrator.distance_data = self._given_assignment(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.target_columns.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.control_columns.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      else:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        self._apply_text_style('failure', '\n
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    def _n_part_split(dataframe, num_of_split, NUM_OF_ITERATION):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      """n_part_split
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      Args:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        dataframe: xxx.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        num_of_split: xxx.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        NUM_OF_ITERATION: xxx.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      """
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      distance_data = pd.DataFrame(columns=['distance'])
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      num_of_pick = len(dataframe.columns) // num_of_split
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      for l in tqdm(range(NUM_OF_ITERATION)):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        col_list = list(dataframe.columns)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        picked_data = pd.DataFrame()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        # random pick
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        picks = []
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        for s in range(num_of_split):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          random_pick = random.sample(col_list, num_of_pick)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          picks.append(random_pick)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          col_list = [i for i in col_list if i not in random_pick]
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        picks[0].extend(col_list)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        for i in range(len(picks)):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          picked_data = pd.concat([
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              picked_data,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              pd.DataFrame(dataframe[picks[i]].sum(axis=1), columns=[i])
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              ], axis=1)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        # calculate distance
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        distance = self._calculate_distance(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            picked_data.reset_index(drop=True)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        distance_data.loc[l, 'distance'] = float(distance)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        for j in range(len(picks)):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          distance_data.at[l, j] = str(sorted(picks[j]))
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      distance_data = (
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          distance_data.drop_duplicates()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          .sort_values('distance')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          .head(3)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          .reset_index(drop=True)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      return distance_data
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    def _find_similar(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        dataframe,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        target_columns,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        num_of_pick_range,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        num_of_covariate,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        ):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      distance_data = pd.DataFrame(columns=['distance'])
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      target_cols = target_columns.replace(', ', ',').split(',')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      # An error occurs when the number of candidates (max num_of_range times
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      # num_of_covariates) is greater than num_of_columns excluding target column.
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      if (
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          len(dataframe.columns) - len(target_cols)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            >= num_of_pick_range[1] * num_of_covariate):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        pass
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      else:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        print('Please check the following:')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        print('* There is something wrong with similarity settings.')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        print('* Total number of columns ー the target = {}'.format(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            len(dataframe.columns) - len(target_cols)))
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        print('* But your settings are {}(max pick#) × {}(covariate#)'.format(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            num_of_pick_range[1], num_of_covariate))
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        print('* Please set it so that it does not exceed.')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        self._apply_text_style('failure', '▲▲▲▲▲▲\n
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    def _from_share(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        dataframe,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        target_share
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        ):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      distance_data = pd.DataFrame(columns=['distance'])
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      combinations = []
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      n = CausalImpactAnalysis.NUM_OF_ITERATION
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      while len(combinations) < CausalImpactAnalysis.COMBINATION_TARGET:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        n -= 1
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        picked_col = np.random.choice(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            dataframe.columns,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            # Shareは50%までなので列数を2分割
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            random.randint(1, len(dataframe.columns)//2 + 1),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            replace=False)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        # (todo)@rhirota シェアを除外済みか全体か検討
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        if float(Decimal(dataframe[picked_col].sum().sum() / dataframe.sum().sum()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        ).quantize(Decimal('0.1'), ROUND_HALF_UP)) == target_share:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          combinations.append(sorted(set(picked_col)))
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        if n == 1:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          self._apply_text_style('failure', '\n
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    def _given_assignment(target_columns, control_columns):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      distance_data = pd.DataFrame(columns=['distance'])
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      distance_data.loc[0, 'distance'] = 0
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      distance_data.loc[0, 0] = str(target_columns.replace(', ', ',').split(','))
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      distance_data.loc[0, 1] = str(control_columns.replace(', ', ',').split(','))
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      return distance_data
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    def _calculate_distance(dataframe):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      total_distance = 0
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      scaled_data = pd.DataFrame()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      for col in dataframe:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        scaled_data[col] = (dataframe[col] - dataframe[col].min()) / (
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            dataframe[col].max() - dataframe[col].min()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      scaled_data = scaled_data.diff().reset_index().dropna()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      for v in itertools.combinations(list(scaled_data.columns), 2):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        distance, _ = fastdtw.fastdtw(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            scaled_data.loc[:, ['index', v[0]]],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            scaled_data.loc[:, ['index', v[1]]],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            dist=euclidean,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        total_distance = total_distance + distance
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      return total_distance
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    def run_experimental_design(self, orchestrator):
\n      return self.experimental_designer.run_experimental_design(self)
\n  
\n\n

In [ ]:
class CausalImpactModel(object):\n\n    def run_causalImpact(self):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      orchestrator.ci_objs = []
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      try:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        self.ci_obj = self.create_causalimpact_object(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.formatted_data,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.date_col_name,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.pre_period_start.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.pre_period_end.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.post_period_start.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.post_period_end.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.num_of_seasons.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.credible_interval.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        orchestrator.ci_objs.append(self.ci_obj)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        self._apply_text_style(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            'success',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            '\n
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    def create_causalimpact_object(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        data,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        date_col,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        pre_start,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        pre_end,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        post_start,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        post_end,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        num_of_seasons,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        credible_interval):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      if data.index.name != date_col: data.set_index(date_col, inplace=True)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      if num_of_seasons == 1:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        causalimpact_object = causalimpact.fit_causalimpact(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            data=data,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            pre_period=(str(pre_start), str(pre_end)),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            post_period=(str(post_start), str(post_end)),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            alpha= 1 - credible_interval / 100,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      else:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        causalimpact_object = causalimpact.fit_causalimpact(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            data=data,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            pre_period=(str(pre_start), str(pre_end)),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            post_period=(str(post_start), str(post_end)),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            alpha= 1 - credible_interval / 100,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            model_options=causalimpact.ModelOptions(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                seasons=[
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    causalimpact.Seasons(num_seasons=num_of_seasons),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                ]
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      return causalimpact_object
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n\n

In [ ]:
class Simulator(object):\n\n    def generate_simulation(self, orchestrator):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      self.test_data = self._extract_data_from_choice(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          orchestrator.your_choice.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          orchestrator.target_col_to_simulate.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          orchestrator.covariate_col_to_simulate.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          orchestrator.formatted_data,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          orchestrator.distance_data,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      self.simulation_params, orchestrator.ci_objs = self._execute_simulation(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          self.test_data,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          orchestrator.date_col_name,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          orchestrator.start_date_value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          orchestrator.end_date_value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          orchestrator.num_of_seasons.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          orchestrator.credible_interval.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          CausalImpactAnalysis.TREAT_DURATION,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          CausalImpactAnalysis.TREAT_IMPACT,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      self._display_simulation_result(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          self.simulation_params,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          orchestrator.ci_objs,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          orchestrator.estimate_icpa.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      self._plot_simulation_result(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          self.simulation_params,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          orchestrator.ci_objs,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          orchestrator.date_col_name,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          orchestrator.tick_count,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          orchestrator.purpose_selection.selected_index,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          orchestrator.credible_interval.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    def _extract_data_from_choice(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        your_choice,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        target_col_to_simulate,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        covariate_col_to_simulate,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        dataframe,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        distance
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        ):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        selection_row = int(your_choice.replace('option_', '')) - 1
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        selection_cols = [
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            [int(t.replace('col_', '')) - 1 for t in list(target_col_to_simulate)],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            [int(t.replace('col_', '')) - 1 for t in list(covariate_col_to_simulate)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            ]]
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        test_data = pd.DataFrame(index = dataframe.index)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        test_column = []
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        for i in selection_cols[0]:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          test_column.extend(eval(distance.at[selection_row,i]))
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        test_data['test'] = dataframe.loc[
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                      :, test_column
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  ].sum(axis=1)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        for col in selection_cols[1]:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          test_data['col_'+ str(col+1)] = dataframe.loc[
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  :, eval(distance.at[selection_row, col])
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              ].sum(axis=1)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        print('* test: {}\n
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    def _execute_simulation(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        dataframe,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        date_col_name,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        start_date_value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        end_date_value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        num_of_seasons,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        credible_interval,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        TREAT_DURATION,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        TREAT_IMPACT,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      ):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      ci_objs = []
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      simulation_params = []
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      adjusted_data = dataframe.copy()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      for duration in tqdm(TREAT_DURATION):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        for impact in tqdm(TREAT_IMPACT, leave=False):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            pre_end_date = end_date_value + datetime.timedelta(days=-duration)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            post_start_date = pre_end_date + datetime.timedelta(days=1)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            adjusted_data.loc[
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                np.datetime64(post_start_date) : np.datetime64(end_date_value),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                'test',] = (
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    dataframe.loc[
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    np.datetime64(post_start_date) : np.datetime64(end_date_value
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    'test',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                ]
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                * impact
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            ci_obj = CausalImpactAnalysis.create_causalimpact_object(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                adjusted_data,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                date_col_name,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                start_date_value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                pre_end_date,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                post_start_date,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                end_date_value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                num_of_seasons,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                credible_interval,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            simulation_params.append([
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                start_date_value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                pre_end_date,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                post_start_date,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                end_date_value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                impact,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                duration,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            ])
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            ci_objs.append(ci_obj)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      return simulation_params, ci_objs
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    def generate_simulation(self, orchestrator):
\n      return self.simulator.generate_simulation(self)
\n  
\n\n

In [ ]:
class Visualizer(object):\n\n      def _trend_check(dataframe, date_col_name, tick_count):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        """trend_check visualize daily trend, 7-day moving average
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        Args:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          dataframe: Wide data to check the trend
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          date_col_name: xxx
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        """
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        df_each = pd.DataFrame(index=dataframe.index)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        col_list = list(dataframe.columns)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        for i in col_list:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          min_max = (
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              dataframe[i] - dataframe[i].min()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              ) / (dataframe[i].max() - dataframe[i].min())
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          df_each = pd.concat([df_each, min_max], axis = 1)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        metric = 'dtw'
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        n_clusters = 5
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        tskm_base = TimeSeriesKMeans(n_clusters=n_clusters, metric=metric,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                                 max_iter=100, random_state=42)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        df_cluster = pd.DataFrame({
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            "pivot": col_list,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            "cluster": tskm_base.fit_predict(df_each.T).tolist()})
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        cluster_counts = (
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            df_cluster["cluster"].value_counts().sort_values(ascending=True))
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        cluster_text = []
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        line_each = []
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        for i in cluster_counts.index:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          clust_list = df_cluster.query("cluster == @i")["pivot"].to_list()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          source = df_each.filter(items=clust_list)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          cluster_text.append(str(clust_list).translate(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              str.maketrans({'[': '', ']': '',  "'": ''})))
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          line_each.append(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              alt.Chart(source.reset_index())
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              .transform_fold(fold=clust_list, as_=['pivot', 'kpi'])
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              .mark_line()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              .encode(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  alt.X(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                      date_col_name + ':T',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                      title=None,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                      axis=alt.Axis(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                          grid=False, format='%Y %b', tickCount=tick_count
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                          ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                      ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  alt.Y('kpi:Q', stack=None, axis=None),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  alt.Color(str(i) + ':N', title=None, legend=None),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  alt.Row(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                      'pivot:N',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                      title=None,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                      header=alt.Header(labelAngle=0, labelAlign='left'),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                      ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              .properties(bounds='flush', height=30)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              .configure_facet(spacing=0)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              .configure_view(stroke=None)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              .configure_title(anchor='end')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        df_long = (
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            pd.melt(dataframe.reset_index(), id_vars=date_col_name)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .groupby(date_col_name)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .sum(numeric_only=True)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .reset_index()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        line_total = (
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            alt.Chart(df_long)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .mark_line()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .encode(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                x=alt.X(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    date_col_name + ':T',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    axis=alt.Axis(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        title='', format='%Y %b', tickCount=tick_count
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                y=alt.Y('value:Q', axis=alt.Axis(title='kpi')),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                color=alt.value('#4285F4'),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        moving_average = (
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            alt.Chart(df_long)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .transform_window(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                rolling_mean='mean(value)',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                frame=[-4, 3],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .mark_line()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .encode(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                x=alt.X(date_col_name + ':T'),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                y=alt.Y('rolling_mean:Q'),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                color=alt.value('#DB4437'),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        tab_total_trend = ipywidgets.Output()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        tab_each_trend = ipywidgets.Output()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        tab_describe_data = ipywidgets.Output()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        tab_result = ipywidgets.Tab(children = [
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            tab_total_trend,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            tab_each_trend,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            tab_describe_data,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            ])
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        tab_result.set_title(0, '>> total_trend')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        tab_result.set_title(1, '>> each_trend')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        tab_result.set_title(2, '>> describe_data')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        display(tab_result)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        with tab_total_trend:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          display(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              (line_total + moving_average).properties(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  width=700,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  height=200,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  title={
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                      'text': ['Daily Trend(blue) & 7days moving average(red)'],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  },
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        with tab_each_trend:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          for i in range(len(cluster_text)):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              print('cluster {}:{}'.format(i, cluster_text[i]))
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              display(line_each[i].properties(width=700))
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        with tab_describe_data:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          display(dataframe.describe(include='all'))
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      def display_causalimpact_result(self, orchestrator):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        print('Test & Control Time Series')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        line = (
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            alt.Chart(orchestrator.formatted_data.reset_index())
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .transform_fold(list(orchestrator.formatted_data.columns))
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .mark_line()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .encode(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                alt.X(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    orchestrator.date_col_name + ':T',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    title=None,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    axis=alt.Axis(format='%Y %b', tickCount=orchestrator.tick_count),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                y=alt.Y('value:Q', axis=alt.Axis(title='kpi')),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                color=alt.Color(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    'key:N',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    legend=alt.Legend(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        title=None,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        orient='none',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        legendY=-20,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        direction='horizontal',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        titleAnchor='start',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    scale=alt.Scale(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        domain=list(orchestrator.formatted_data.columns),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        range=CausalImpactAnalysis.colors,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .properties(height=200, width=600)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        rule = (
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            alt.Chart(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              pd.DataFrame({
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                'Date': [
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    str(orchestrator.post_period_start.value),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    str(orchestrator.post_period_end.value)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    ],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                'color': ['red', 'orange'],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                })
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .mark_rule(strokeDash=[5, 5])
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .encode(x='Date:T', color=alt.Color('color:N', scale=None))
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        display((line+rule).properties(height=200, width=600))
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        print('=' * 100)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        self.plot_causalimpact(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.ci_objs[0],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.pre_period_start.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.pre_period_end.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.post_period_start.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.post_period_end.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.credible_interval.value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.date_col_name,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.tick_count,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            orchestrator.purpose_selection.selected_index
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      def plot_causalimpact(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          causalimpact_object,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          pre_start,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          pre_end,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          tread_start,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          treat_end,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          credible_interval,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          date_col_name,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          tick_count,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          purpose_selection
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        ):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        causalimpact_df = causalimpact_object.series#.copy()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        mape = mean_absolute_percentage_error(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            causalimpact_df['observed'][str(pre_start) : str(pre_end)],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            causalimpact_df['posterior_mean'][str(pre_start) : str(pre_end)],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        threshold = round(1 - credible_interval / 100, 2)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        line_1 = (
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            alt.Chart(causalimpact_df.reset_index())
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .transform_fold([
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                'observed',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                'posterior_mean',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            ])
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .mark_line()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .encode(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                x=alt.X(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    'yearmonthdate(' + date_col_name + ')',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    axis=alt.Axis(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        title='',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        labels=False,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        ticks=False,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        format='%Y %b',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        tickCount=tick_count,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                y=alt.Y(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    'value:Q',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    scale=alt.Scale(zero=False),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    axis=alt.Axis(title=''),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                color=alt.Color(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    'key:N',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    legend=alt.Legend(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        title=None,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        orient='none',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        legendY=-20,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        direction='horizontal',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        titleAnchor='start',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    sort=['posterior_mean', 'observed'],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                strokeDash=alt.condition(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    alt.datum.key == 'posterior_mean',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    alt.value([5, 5]),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    alt.value([0]),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        area_1 = (
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            alt.Chart(causalimpact_df.reset_index())
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .mark_area(opacity=0.3)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .encode(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                x=alt.X('yearmonthdate(' + date_col_name + ')'),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                y=alt.Y('posterior_lower:Q', scale=alt.Scale(zero=False)),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                y2=alt.Y2('posterior_upper:Q'),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        line_2 = (
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            alt.Chart(causalimpact_df.reset_index())
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .mark_line(strokeDash=[5, 5])
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .encode(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                x=alt.X(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    'yearmonthdate(' + date_col_name + ')',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    axis=alt.Axis(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        title='',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        labels=False,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        ticks=False,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        format='%Y %b',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        tickCount=tick_count,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                y=alt.Y(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    'point_effects_mean:Q',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    scale=alt.Scale(zero=False),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    axis=alt.Axis(title=''),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        area_2 = (
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            alt.Chart(causalimpact_df.reset_index())
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .mark_area(opacity=0.3)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .encode(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                x=alt.X('yearmonthdate(' + date_col_name + ')'),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                y=alt.Y('point_effects_lower:Q', scale=alt.Scale(zero=False)),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                y2=alt.Y2('point_effects_upper:Q'),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        line_3 = (
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            alt.Chart(causalimpact_df.reset_index())
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .mark_line(strokeDash=[5, 5])
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .encode(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                x=alt.X(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    'yearmonthdate(' + date_col_name + ')',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    axis=alt.Axis(title='', format='%Y %b', tickCount=tick_count),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                y=alt.Y(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    'cumulative_effects_mean:Q',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    scale=alt.Scale(zero=False),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    axis=alt.Axis(title=''),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                ),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        area_3 = (
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            alt.Chart(causalimpact_df.reset_index())
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .mark_area(opacity=0.3)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .encode(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                x=alt.X(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    'yearmonthdate(' + date_col_name + ')',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    axis=alt.Axis(title='')),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                y=alt.Y('cumulative_effects_lower:Q', scale=alt.Scale(zero=False),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        axis=alt.Axis(title='')),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                y2=alt.Y2('cumulative_effects_upper:Q'),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        zero_line = (
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            alt.Chart(pd.DataFrame({'y': [0]}))
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .mark_rule()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .encode(y='y', color=alt.value('gray'))
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        rules = (
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            alt.Chart(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                pd.DataFrame({
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    'Date': [str(tread_start), str(treat_end)],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    'color': ['red', 'orange'],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                })
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .mark_rule(strokeDash=[5, 5])
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            .encode(x='Date:T', color=alt.Color('color:N', scale=None))
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        watermark = alt.Chart(pd.DataFrame([1])).mark_text(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            align='center',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            dx=0,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            dy=0,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            fontSize=48,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            text='mock experiment',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            color='red'
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          ).encode(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            opacity=alt.value(0.5)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        if purpose_selection == 1:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          cumulative = line_3 + area_3 + rules + zero_line + watermark
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        elif causalimpact_object.summary.p_value.average >= threshold:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          cumulative = area_3 + rules + zero_line
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        else:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          cumulative = line_3 + area_3 + rules + zero_line
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        plot = alt.vconcat(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            (line_1 + area_1 + rules).properties(height=100, width=600),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            (line_2 + area_2 + rules + zero_line).properties(height=100, width=600),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            (cumulative).properties(height=100, width=600),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        tab_data = ipywidgets.Output()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        tab_report = ipywidgets.Output()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        tab_summary = ipywidgets.Output()
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        tab_result = ipywidgets.Tab(children = [tab_summary, tab_report, tab_data])
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        tab_result.set_title(0, '>> summary')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        tab_result.set_title(1, '>> report')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        tab_result.set_title(2, '>> data')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        with tab_summary:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          print('Approximate model accuracy >> MAPE:{:.2%}'.format(mape))
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          if mape <= 0.05:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              self._apply_text_style(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  'success',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  'Very Good: The difference between actual and predicted values ​​is slight.')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          elif mape <= 0.10:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              self._apply_text_style(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  'success',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  'Good: The difference between the actual and predicted values ​​is within the acceptable range.')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          elif mape <= 0.15:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              self._apply_text_style(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  'failure',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  'Medium: he difference between the actual and predicted values ​​ismoderate, so this is only a reference value.')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          else:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              self._apply_text_style(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  'failure',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  'Bad: The difference between actual and predicted values ​​is large, so we do not recommend using it.')
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          if causalimpact_object.summary.p_value.average <= threshold:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              self._apply_text_style('success', f'
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      def _visualize_candidate(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          dataframe,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          distance_data,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          start_date_value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          end_date_value,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          date_col_name,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          tick_count
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          ):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        self._apply_text_style(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              'failure',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              '
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      def _display_simulation_result(simulation_params, ci_objs, estimate_icpa):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          simulation_df = pd.DataFrame(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              index=[],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              columns=[
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  'mock_lift',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  'Days_simulated',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  'Pre_Period_MAPE',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  'Post_Period_MAPE',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  'Total_effect',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  'Average_effect',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  'Required_budget',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  'p_value',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                  'predicted_lift'
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              ],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          for i in range(len(ci_objs)):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            impact_df = ci_objs[i].series
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            impact_dict = {
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                'test_period':'('+str(simulation_params[i][5])+'d) '+str(simulation_params[i][2])+'~'+str(simulation_params[i][3]),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                'mock_lift_rate': simulation_params[i][4] - 1,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                'predicted_lift_rate': ci_objs[i].summary.loc['average', 'rel_effect'],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                'Days_simulated': simulation_params[i][5],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                'Pre_Period_MAPE': [
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    mean_absolute_percentage_error(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        impact_df.loc[:, 'observed'][
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                            str(simulation_params[i][0]) : str(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                                simulation_params[i][1]
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                            )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        ],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        impact_df.loc[:, 'posterior_mean'][
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                            str(simulation_params[i][0]) : str(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                                simulation_params[i][1]
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                            )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        ],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                ],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                'Post_Period_MAPE': [
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    mean_absolute_percentage_error(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        impact_df.loc[:, 'observed'][
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                            str(simulation_params[i][2]) : str(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                                simulation_params[i][3]
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                            )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        ],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        impact_df.loc[:, 'posterior_mean'][
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                            str(simulation_params[i][2]) : str(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                                simulation_params[i][3]
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                            )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        ],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                ],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                'Total_effect': ci_objs[i].summary.loc['cumulative', 'abs_effect'],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                'Average_effect': ci_objs[i].summary.loc['average', 'abs_effect'],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                'Required_budget': [
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    ci_objs[i].summary.loc['cumulative', 'abs_effect'] * estimate_icpa
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                ],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                'p_value': ci_objs[i].summary.loc['average', 'p_value'],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            }
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            simulation_df = pd.concat(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                [simulation_df, pd.DataFrame.from_dict(impact_dict)],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                ignore_index=True,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          display(self._apply_text_style(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                18,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                'A/A Test: Check the error without intervention'))
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          print('> If p_value < 0.05, please suspect "poor model accuracy"(See Pre_Period_MAPE) or "data drift"(See Time Series Chart).
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      def _plot_simulation_result(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          simulation_params,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          ci_objs,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          date_col_name,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          tick_count,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          purpose_selection,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          credible_interval,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          ):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        mock_combinations = []
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        for i in range(len(simulation_params)):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          mock_combinations.append(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                [
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                    '{}d:+{:.0%}'.format(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        simulation_params[i][5],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                        simulation_params[i][4]-1)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                ])
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        simulation_tb=[ipywidgets.Output() for tab in mock_combinations]
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        tab_simulation = ipywidgets.Tab(simulation_tb)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n        for id,name in enumerate(mock_combinations):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          tab_simulation.set_title(id,name)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          with simulation_tb[id]:
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n            print(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n                'Pre Period:{} ~ {}
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    def display_causalimpact_result(self, orchestrator):
\n      return self.visualizer.display_causalimpact_result(self)
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n    def _generate_choice(self):
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      self.your_choice = ipywidgets.Dropdown(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          options=['option_1', 'option_2', 'option_3'],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          description='your choice:',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      self.target_col_to_simulate = ipywidgets.SelectMultiple(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          options=['col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6'],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          description='target col:',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          value=['col_1',],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      self.covariate_col_to_simulate = ipywidgets.SelectMultiple(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          options=['col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6'],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          description='covatiate col:',
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          value=['col_2',],
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          style={'description_width': 'initial'},
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      display(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          self._apply_text_style(
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              18,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              '⑷ Please select option, test column & control column(s).'),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          ipywidgets.HBox([
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              self.your_choice,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              self.target_col_to_simulate,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n              self.covariate_col_to_simulate,
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n          ]),
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n      )
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n  
\n\n

In [ ]:
# @title Step.2

%%time

case_1.load_data()

case_1.format_data()

dict_params = PreProcess.saving_params(case_1)



if case_1.purpose_selection.selected_index == 0:

  case_1.run_causalImpact()

else:

  case_1.run_experimental_design()



print('\nExecution datetime(GMT):{}'.format(datetime.datetime.now()))\n\n

In [ ]:
# @title Step.3

%%time

if case_1.purpose_selection.selected_index == 0:

  case_1.display_causalimpact_result()

else:

  case_1.generate_simulation()



print('\nExecution datetime(GMT):{}'.format(datetime.datetime.now()))\n\n

# (Optional) Case_2

In [ ]:
# @title Case_2 Step.1

overwrite_pramas = True #@param {type:"boolean"}

case_2 = CausalImpactAnalysis()

case_2.generate_ui()

if overwrite_pramas == True: PreProcess.set_params(case_2, dict_params)\n\n

In [ ]:
# @title Case_2 Step.2

%%time

case_2.load_data()

case_2.format_data()



if case_2.purpose_selection.selected_index == 0:

  case_2.run_causalImpact()

else:

  case_2.run_experimental_design()



print('\nExecution datetime(GMT):{}'.format(datetime.datetime.now()))\n\n

In [ ]:
# @title Case_2 Step.3

%%time

if case_2.purpose_selection.selected_index == 0:

  case_2.display_causalimpact_result()

else:

  case_2.generate_simulation()



print('\nExecution datetime(GMT):{}'.format(datetime.datetime.now()))\n\n

# (Optional) Case_3

In [ ]:
# @title Case_3 Step.1

overwrite_pramas = False #@param {type:"boolean"}

case_3 = CausalImpactAnalysis()

case_3.generate_ui()

if overwrite_pramas == True: PreProcess.set_params(case_3, dict_params)\n\n

In [ ]:
# @title Case_3 Step.2

%%time

case_3.load_data()

case_3.format_data()



if case_3.purpose_selection.selected_index == 0:

  case_3.run_causalImpact()

else:

  case_3.run_experimental_design()



print('\nExecution datetime(GMT):{}'.format(datetime.datetime.now()))\n\n

In [ ]:
# @title Case_3 Step.3

%%time

if case_3.purpose_selection.selected_index == 0:

  case_3.display_causalimpact_result()

else:

  case_3.generate_simulation()



print('\nExecution datetime(GMT):{}'.format(datetime.datetime.now()))\n\n